# Fashion MNIST 경진대회

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# 실행할 때마다 같은 결과를 출력하기 위한 seed 값 설정
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 전처리

In [37]:
from tensorflow.keras.datasets import fashion_mnist
(X_train, Y_class_train), (X_test, Y_class_test) = fashion_mnist.load_data()

### Fashion MNIST ver.0

In [4]:
# 0 ~ 1 사이의 값으로 정규화
X_train = X_train.reshape(-1, 784)
X_train = X_train / 255.0
X_train.shape

(60000, 784)

In [5]:
X_test = X_test.reshape(-1, 784)
X_test = X_test / 255.0
type(X_test[0,0])

numpy.float64

In [6]:
# One hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_class_train, 10)
Y_test = to_categorical(Y_class_test, 10)
Y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

### 모델 정의, 설정, 학습

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import os
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [8]:
model = Sequential([
    Dense(512, input_dim=784, activation='relu'),
    Dense(5, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
Total params: 404,545
Trainable params: 404,545
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 모델 실행환경 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# 모델 저장 조건 설정
modelpath = MODEL_DIR + "best{epoch:03d}-{val_loss:.4f}.hdf5"

In [11]:
# 모델 최적화 설정
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [12]:
# 모델 실행
history = model.fit(X_train, Y_train, validation_split=0.2,
                    epochs=30, batch_size=200, verbose=0,
                    callbacks=[checkpointer, early_stopping])


Epoch 00001: val_loss improved from inf to 1.50084, saving model to ./model/best001-1.5008.hdf5

Epoch 00002: val_loss improved from 1.50084 to 1.30088, saving model to ./model/best002-1.3009.hdf5

Epoch 00003: val_loss improved from 1.30088 to 0.97617, saving model to ./model/best003-0.9762.hdf5

Epoch 00004: val_loss improved from 0.97617 to 0.71903, saving model to ./model/best004-0.7190.hdf5

Epoch 00005: val_loss improved from 0.71903 to 0.59269, saving model to ./model/best005-0.5927.hdf5

Epoch 00006: val_loss improved from 0.59269 to 0.53268, saving model to ./model/best006-0.5327.hdf5

Epoch 00007: val_loss improved from 0.53268 to 0.49168, saving model to ./model/best007-0.4917.hdf5

Epoch 00008: val_loss improved from 0.49168 to 0.46411, saving model to ./model/best008-0.4641.hdf5

Epoch 00009: val_loss improved from 0.46411 to 0.44729, saving model to ./model/best009-0.4473.hdf5

Epoch 00010: val_loss improved from 0.44729 to 0.41966, saving model to ./model/best010-0.4197

In [15]:
from tensorflow.keras.models import load_model
best_model = load_model('./model/best021-0.3578.hdf5')

In [16]:
# 테스트 정확도 출력 
acc = model.evaluate(X_test, Y_test, verbose=2)[1]
print(f"\n Test Accuracy: {acc:.4f}")

313/313 - 1s - loss: 0.4046 - accuracy: 0.8821

 Test Accuracy: 0.8821


### Fashion MNIST ver.1 (Conv2D 2개)
- Conv2D 2개

In [18]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [19]:
# 0 ~ 1 사이의 값으로 정규화
X_train = X_train / 255.0
X_test = X_test / 255.0

In [20]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    Flatten(),             
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               18874880  
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 18,898,826
Trainable params: 18,898,826
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 모델 실행환경 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
# 모델 저장 조건 설정
modelpath = MODEL_DIR + "best{epoch:03d}-{val_loss:.4f}.hdf5"

In [23]:
# 모델 최적화 설정
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [24]:
# 모델 실행
history = model.fit(X_train, Y_train, validation_split=0.2,
                    epochs=30, batch_size=200, verbose=0,
                    callbacks=[checkpointer, early_stopping])


Epoch 00001: val_loss improved from inf to 0.29721, saving model to ./model/best001-0.2972.hdf5

Epoch 00002: val_loss improved from 0.29721 to 0.26376, saving model to ./model/best002-0.2638.hdf5

Epoch 00003: val_loss improved from 0.26376 to 0.24976, saving model to ./model/best003-0.2498.hdf5

Epoch 00004: val_loss improved from 0.24976 to 0.24289, saving model to ./model/best004-0.2429.hdf5

Epoch 00005: val_loss did not improve from 0.24289

Epoch 00006: val_loss did not improve from 0.24289

Epoch 00007: val_loss did not improve from 0.24289

Epoch 00008: val_loss did not improve from 0.24289

Epoch 00009: val_loss did not improve from 0.24289

Epoch 00010: val_loss did not improve from 0.24289

Epoch 00011: val_loss did not improve from 0.24289

Epoch 00012: val_loss did not improve from 0.24289

Epoch 00013: val_loss did not improve from 0.24289

Epoch 00014: val_loss did not improve from 0.24289


In [25]:
from tensorflow.keras.models import load_model
best_model = load_model('./model/best004-0.2429.hdf5')

In [26]:
# 테스트 정확도 출력 
acc = model.evaluate(X_test, Y_test, verbose=2)[1]
print(f"\n Test Accuracy: {acc:.4f}")

313/313 - 2s - loss: 0.4915 - accuracy: 0.9089

 Test Accuracy: 0.9089


### Fashion MNIST ver.2 (Conv2D 2개 +MaxPooling 1개)
- Conv2D 2개
- MaxPooling 1개

In [38]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [39]:
# 0 ~ 1 사이의 값으로 정규화
X_train = X_train / 255.0
X_test = X_test / 255.0

In [40]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),             
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               4719104   
_________________________________________________________________
dense_8 (Dense)              (None, 10)                5130      
Total params: 4,743,050
Trainable params: 4,743,050
Non-trainable params: 0
____________________________________________

In [41]:
# 모델 실행환경 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
# 모델 저장 조건 설정
modelpath = MODEL_DIR + "best{epoch:03d}-{val_loss:.4f}.hdf5"

In [43]:
# 모델 최적화 설정
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [44]:
# 모델 실행
history = model.fit(X_train, Y_train, validation_split=0.2,
                    epochs=30, batch_size=200, verbose=0,
                    callbacks=[checkpointer, early_stopping])


Epoch 00001: val_loss improved from inf to 0.31296, saving model to ./model/best001-0.3130.hdf5

Epoch 00002: val_loss improved from 0.31296 to 0.26875, saving model to ./model/best002-0.2688.hdf5

Epoch 00003: val_loss improved from 0.26875 to 0.25079, saving model to ./model/best003-0.2508.hdf5

Epoch 00004: val_loss improved from 0.25079 to 0.23139, saving model to ./model/best004-0.2314.hdf5

Epoch 00005: val_loss improved from 0.23139 to 0.22386, saving model to ./model/best005-0.2239.hdf5

Epoch 00006: val_loss did not improve from 0.22386

Epoch 00007: val_loss did not improve from 0.22386

Epoch 00008: val_loss did not improve from 0.22386

Epoch 00009: val_loss did not improve from 0.22386

Epoch 00010: val_loss did not improve from 0.22386

Epoch 00011: val_loss did not improve from 0.22386

Epoch 00012: val_loss did not improve from 0.22386

Epoch 00013: val_loss did not improve from 0.22386

Epoch 00014: val_loss did not improve from 0.22386

Epoch 00015: val_loss did not 

In [45]:
from tensorflow.keras.models import load_model
best_model = load_model('./model/best005-0.2239.hdf5')

In [46]:
# 테스트 정확도 출력 
acc = model.evaluate(X_test, Y_test, verbose=2)[1]
print(f"\n Test Accuracy: {acc:.4f}")

313/313 - 1s - loss: 0.4359 - accuracy: 0.9159

 Test Accuracy: 0.9159


### Fashion MNIST ver.3 (MaxPooling 1개 + Dropout 2개)
- Conv2D 2개
- MaxPooling 1개
- Dropout 2개

In [28]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [29]:
# 0 ~ 1 사이의 값으로 정규화
X_train = X_train / 255.0
X_test = X_test / 255.0

In [30]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    Flatten(),             
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               4719104   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)              

In [31]:
# 모델 실행환경 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# 모델 저장 조건 설정
modelpath = MODEL_DIR + "best{epoch:03d}-{val_loss:.4f}.hdf5"

In [33]:
# 모델 최적화 설정
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [34]:
# 모델 실행
history = model.fit(X_train, Y_train, validation_split=0.2,
                    epochs=30, batch_size=200, verbose=0,
                    callbacks=[checkpointer, early_stopping])


Epoch 00001: val_loss improved from inf to 0.31600, saving model to ./model/best001-0.3160.hdf5

Epoch 00002: val_loss improved from 0.31600 to 0.27383, saving model to ./model/best002-0.2738.hdf5

Epoch 00003: val_loss improved from 0.27383 to 0.24451, saving model to ./model/best003-0.2445.hdf5

Epoch 00004: val_loss improved from 0.24451 to 0.23184, saving model to ./model/best004-0.2318.hdf5

Epoch 00005: val_loss improved from 0.23184 to 0.22887, saving model to ./model/best005-0.2289.hdf5

Epoch 00006: val_loss improved from 0.22887 to 0.21123, saving model to ./model/best006-0.2112.hdf5

Epoch 00007: val_loss improved from 0.21123 to 0.20048, saving model to ./model/best007-0.2005.hdf5

Epoch 00008: val_loss did not improve from 0.20048

Epoch 00009: val_loss did not improve from 0.20048

Epoch 00010: val_loss did not improve from 0.20048

Epoch 00011: val_loss did not improve from 0.20048

Epoch 00012: val_loss did not improve from 0.20048

Epoch 00013: val_loss did not improv

In [35]:
from tensorflow.keras.models import load_model
best_model = load_model('./model/best007-0.2005.hdf5')

In [36]:
# 테스트 정확도 출력 
acc = model.evaluate(X_test, Y_test, verbose=2)[1]
print(f"\n Test Accuracy: {acc:.4f}")

313/313 - 2s - loss: 0.2642 - accuracy: 0.9285

 Test Accuracy: 0.9285
